# Import Modules

In [104]:
import json

import numpy as np
import pandas as pd
import torch

import utils

# Set Vegetable

In [113]:
vegetable = utils.VEGETABLES[15]

json_open = open('./free_params.json', 'r')
params = json.load(json_open)
param = params[vegetable]

print(f"vegetable: {vegetable}")
print(f"free params: {param}")

vegetable: ミニトマト
free params: {'T': 10, 'batch_size': 16, 'num_epochs': 70, 'learning_rate': 0.001, 'weight_decay': 0.001, 'eps': 1e-08, 'hidden_size': 500, 'dropout_ratio': 0.5, 'num_ensamble': 10}


# Load Data

In [108]:
train_test = pd.read_csv("./data/mapped_train_test.csv")
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y-%m-%d")
weather = pd.read_csv("./data/sorted_mapped_adjusted_weather.csv")
train_test = pd.concat([train_test, weather], axis=1)
train_test = train_test.reset_index(drop=True)

train_test["year"] = train_test.date.dt.year
years = pd.get_dummies(train_test["year"])
train_test = train_test.drop(columns="year")
train_test = pd.concat([train_test, years], axis=1)

train_test["month"] = train_test.date.dt.month
months = pd.get_dummies(train_test["month"])
train_test = train_test.drop(columns="month")
train_test = pd.concat([train_test, months], axis=1)

areas = pd.get_dummies(train_test["area"])
train_test = train_test.drop(columns="area")
train_test = pd.concat([train_test, areas], axis=1)

test_size = pd.read_csv("./data/test.csv").shape[0]
train = train_test[:-test_size]
test = train_test[-test_size:]

train = utils.get_target_values(train, vegetable)
test = utils.get_target_values(test, vegetable)

changed_col = [1, 0] + [i for i in np.arange(2, train.shape[1])]
train = train[:, changed_col]
test = test[:, changed_col]

test_amount = pd.read_csv("./data/test_amount.csv")
test_amount = test_amount[vegetable].values
test[:, 1] = test_amount

train_loader, train, test, ss = utils.preprocess_data_submit(train, test, T=param["T"], batch_size=param["batch_size"])

# Assign Free Params

In [109]:
future = test.shape[0]
num_ensamble = param["num_ensamble"]
num_epochs = param["num_epochs"]
learning_rate = param["learning_rate"]
weight_decay = param["weight_decay"]
eps = param["eps"]
hidden_size = param["hidden_size"]
dropout_ratio = param["dropout_ratio"]

# Training

In [110]:
models = []
for _ in range(num_ensamble):
    
    model = utils.pipeline_rnn_submit(train_loader, train, test, future=future, num_epochs=num_epochs, lr=learning_rate,
                                      weight_decay=weight_decay, eps=eps, hidden_size=hidden_size, dropout_ratio=dropout_ratio)
    models.append(model)

# Predict

In [111]:
preds = []

for model in models:
    with torch.no_grad():
        model.eval()
        pred_y = model.predict(train, test, future)

    pred_y = pred_y.cpu().detach().numpy()
    pred_y = np.concatenate([pred_y.reshape(-1, 1), test[:, 1:]], axis=1)
    pred_y[:, :7] = ss.inverse_transform(pred_y[:, :7])
    pred_y = pred_y[:, 0].tolist()

    preds.append(pred_y)

pred_y = np.mean(np.array(preds),axis=0)

In [112]:
pred_y.tolist()

[117.58034579990439,
 115.10439185069033,
 106.84217198725442,
 105.10640037504405,
 103.08203141566896,
 104.3724802870103,
 103.2320974909624,
 105.39044823418752,
 106.4624123055404,
 105.2411926827422,
 106.83260997632263,
 104.4700925528698,
 105.27614543313469,
 107.44235181009597,
 108.05174828282823,
 109.95821038695894,
 116.49819389702705,
 119.03017368470537,
 120.93572578483595,
 117.54971995734272]